# Challenge: Mission to Mars

## Step 1 - Scraping

### NASA Mars News

#### Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [1]:
#Import dependencies.
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver

#### Web Scraping

In [2]:
#First notate the URL that we will scrape the information from.
urlNews = "https://mars.nasa.gov/news"

#Retrieve page with splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome')

#Create BeautifulSoup object; parse with 'html.parser'
browser.visit(urlNews)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

#Discover and assign to variables the latest "News Title" & "Paragraph Text"
news_title = soup.find('div',class_='content_title').a.text.strip()
news_p = soup.find('div', class_='rollover_description_inner').text.strip()

In [3]:
news_title

"NASA's Opportunity Rover Mission on Mars Comes to End"

In [4]:
news_p

"NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet."

### JPL Mars Space Images - Featured Image

#### * Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called `featured_image_url`.

#### * Make sure to find the image url to the full size `.jpg` image.

#### * Make sure to save a complete url string for this image.

In [5]:
urlImage = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome')

browser.visit(urlImage)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# scrape the html code
featured_image_url = soup.find('div', class_='carousel_items')

#dig into the html code, and split it until we're able to isolate the image url pathway only.
featured_image_url = featured_image_url.article["style"].split("(")[1]
featured_image_url = featured_image_url.split(")")[0]
featured_image_url = featured_image_url.split("'")[1]
featured_image_url = featured_image_url.split("'")[0]

#attach the pathway to the end of a url pathway to complete URL assembly.
featured_image_url = "https://www.jpl.nasa.gov"+featured_image_url

In [6]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18914-1920x1200.jpg'

### Mars Weather

#### Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called `mars_weather`.

In [9]:
# URL
urlWeather = "https://twitter.com/marswxreport?lang=en"

response = requests.get(urlWeather)
soup = BeautifulSoup(response.text, "html.parser")

# Scrape weather data from tweet
mars_weather = soup.find('p', class_="TweetTextSize").text

# Strip off the "InSight" from the weather data string and capitalize "sol" to "Sol"
mars_weather = mars_weather.lstrip('InSight ')
mars_weather = mars_weather.capitalize()
mars_weather = mars_weather.split('pic.twitter.com')[0]

In [10]:
mars_weather

'Sol 400 (2020-01-11) low -99.1ºc (-146.5ºf) high -15.7ºc (3.8ºf)\nwinds from the sse at 5.5 m/s (12.3 mph) gusting to 22.3 m/s (49.9 mph)\npressure at 6.40 hpa'

### Mars Facts

#### * Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

#### * Use Pandas to convert the data to a HTML table string.

In [11]:
# URL
urlFacts = "https://space-facts.com/mars/"

# Use Pandas to scrape data.
marsScrape = pd.read_html(urlFacts)

# Convert data into dataframe table, and rename the columns & assign index.
marsScrape_df = marsScrape[0]
marsScrape_df.columns = ['Measurement','Value']
marsScrape_df = marsScrape_df.set_index('Measurement')
marsScrape_df

,Value
Measurement,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [12]:
# Convert dataframe into a table and make some minor adjustments to smooth out html integration.
mars_htmlTable = marsScrape_df.to_html()
mars_htmlTable = mars_htmlTable.replace("\n","")
mars_htmlTable = mars_htmlTable.replace("right","left")
mars_htmlTable

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: left;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Measurement</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

#### * Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

#### * You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

#### * Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys `img_url` and `title`.

#### * Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [13]:
urlHemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

response = requests.get(urlHemisphere)
soup = BeautifulSoup(response.text, "html.parser")

scrape = soup.find_all('div', class_="item")

path = []

# iterate through the variable scrape that contains BS4 scraped info.
# iterated action: for the value within the <a> tag and href section, append into path array.
for i in scrape:
    path.append(i.a['href'])
path

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [14]:
hemisphere_image_urls = []
urlHemi = "https://astrogeology.usgs.gov"

# iterate through the path array and scrape each pathway into a variable called "soup".
# each iteration will extract the title and img url within the soup...
# ...and then iterate into hemisphere_image_url array as a nested dictionary!
for i in path:
    response = requests.get(f"{urlHemi}{i}")
    soup = BeautifulSoup(response.text, "html.parser")
    title = soup.find('h2', class_='title').text.split("Enhanced")[0]
    img_url = soup.find_all('li')[1].a['href']
    hemisphere_image_urls.append({"title":title,"img_url":img_url})

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]